# CM2000 Lab 2.C. Homography

This code was adapted from OpenCV tutorials.

<a target="_blank" href="https://learnopencv.com">View on learnopencv.com</a>

## Imports

In [1]:
import cv2
import os
import sys
import numpy as np
from functools import partial
import matplotlib.pyplot as plt
from tools import *

## Task 1: Homography of an image


### 1.1. Code assignment

Run homography for a frame of a hockey and amrican football play. Save your results as './homography/homography_hockey.jpg' and './homography/homography_football.jpg'.  
Then, choose another sport where you think the same technique could work, find a picture of the field and a frame of a play, and perform the homography. Save your results as './homography/homography_sport.jpg'.

In [2]:
# Globals

drawing = False  # true if mouse is pressed
src_x, src_y = -1, -1
dst_x, dst_y = -1, -1

src_list = []
dst_list = []

# Define the frame and field file paths for the homography
frame_path = './homography/frame_hockey.jpg'
field_path = './homography/field_hockey.jpg'
homography_path = './homography/homography_hockey.jpg'


In [3]:
# Mouse callback functions
def select_points_src(event, x, y, flags, param):
    '''mouse callback function'''
    global src_x, src_y, drawing
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        src_x, src_y = x, y
        cv2.circle(src_copy, (x, y), 5, (0, 0, 255), -1)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False

def select_points_dst(event, x, y, flags, param):
    '''mouse callback function'''
    global dst_x, dst_y, drawing
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        dst_x, dst_y = x, y
        cv2.circle(dst_copy, (x, y), 5, (0, 0, 255), -1)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False

In [4]:
# Merging functions
def get_plan_view(src, dst, src_list, dst_list):
    src_pts = np.array(src_list).reshape(-1, 1, 2)
    dst_pts = np.array(dst_list).reshape(-1, 1, 2)
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    print("H:")
    print(H)
    plan_view = cv2.warpPerspective(src, H, (dst.shape[1], dst.shape[0]))
    return plan_view


def merge_views(src, dst, src_list, dst_list):
    plan_view = get_plan_view(src, dst, src_list, dst_list)
    for i in range(0, dst.shape[0]):
        for j in range(0, dst.shape[1]):
            if (plan_view.item(i, j, 0) == 0 and
               plan_view.item(i, j, 1) == 0 and
               plan_view.item(i, j, 2) == 0):
                plan_view.itemset((i, j, 0), dst.item(i, j, 0))
                plan_view.itemset((i, j, 1), dst.item(i, j, 1))
                plan_view.itemset((i, j, 2), dst.item(i, j, 2))
    return plan_view

When you run the code below, you will be prompted with 2 windows: the field and the frame.  
Mark a point in the video, then mark the same point in the field, and press key 's' to save the point to the homography. The code "remembers" only the last point pressed at each window when performing the save operation.  
Repeat for at least 4 points, then press key 'm' to perform the homography.  
Then, press key 'q' to quit and release resources. Wait a few seconds before running the next cell.  

In [5]:
# Homography for an image
print("Mark a point in the video, then mark the same point in the field, and press key s")
print("Repeat for at least 4 points.")

src = cv2.imread(frame_path, -1)
src_copy = src.copy()
cv2.namedWindow('src')
cv2.moveWindow("src", 80, 80)
cv2.setMouseCallback('src', select_points_src)

dst = cv2.imread(field_path, -1)
dst_copy = dst.copy()
cv2.namedWindow('dst')
cv2.moveWindow("dst", 780, 80)
cv2.setMouseCallback('dst', select_points_dst)

while True:
    cv2.imshow('src', src_copy)
    cv2.imshow('dst', dst_copy)
    k = cv2.waitKey(1) & 0xFF
    if k == ord('s'):
        print('...saving points')
        cv2.circle(src_copy, (src_x, src_y), 5, (0, 255, 0), -1)
        cv2.circle(dst_copy, (dst_x, dst_y), 5, (0, 255, 0), -1)
        src_list.append([src_x, src_y])
        dst_list.append([dst_x, dst_y])
        print("src points:")
        print(src_list)
        print("dst points:")
        print(dst_list)

        if len(src_list) >= 4 and len(dst_list) >= 4:
            print("Press m to merge.")
    elif k == ord('m'):
        print('...merging views')
        merge = merge_views(src, dst, src_list, dst_list)
        cv2.imwrite(homography_path, merge)
        cv2.imshow("merge", merge)
        print("Press q to quite and release resources.")
    elif k == 27 or k == ord('q'):
        break

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

Mark a point in the video, then mark the same point in the field, and press key s
Repeat for at least 4 points.
...saving points
src points:
[[1699, 817]]
dst points:
[[562, 96]]
...saving points
src points:
[[1699, 817], [851, 954]]
dst points:
[[562, 96], [476, 136]]
...saving points
src points:
[[1699, 817], [851, 954], [854, 494]]
dst points:
[[562, 96], [476, 136], [475, 8]]
...saving points
src points:
[[1699, 817], [851, 954], [854, 494], [1833, 481]]
dst points:
[[562, 96], [476, 136], [475, 8], [594, 10]]
Press m to merge.
...merging views
H:
[[ 7.00464539e-01  1.00915288e+00  1.36703243e+02]
 [ 2.49445866e-02  1.00812757e+00 -4.98544280e+02]
 [ 6.53228990e-04  2.10287297e-03  1.00000000e+00]]
Press q to quite and release resources.


-1

### 1.2. Written assignment

Try using different points in the field to perform the homography. Identify the limitations and write a paragraph with your reflections.

*answer here*

## Task 2: Homography of a video


### 2.1. Code assignment

Run the code to use the homography matrix computed for the first frame of the video and perform the homography for the whole video. Do it for both hockey and football.  
Save your results as './homography/homography_hockey_static.gif' and './homography/homography_football_static.gif'.

In [9]:
# Define the video path for the homography
video_path = './homography/video_hockey.mp4'
homography_name = './homography/homography_hockey_static.gif'


When you run the code below, you will be prompted with a window showing the result of the homography in slow motion.   
Wait for the animation to finish. Then, press key 'q' to quit and release resources. Wait a few seconds before running the next cell.  

In [7]:
print('...Merging views')
# Store merged frames
merged_frames = []
# Read video
video = cv2.VideoCapture(video_path)
# Get frame count and rate
frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = int(video.get(cv2.CAP_PROP_FPS))
# Read field image
dst = cv2.imread(field_path, -1)
print("Wait for the animation of the tracking. Then press Enter to finish.")

i=0
while True:
    # Exit if video not opened.
    if not video.isOpened():
        print ("Could not open video")
        sys.exit()

    # Read video frame.
    ok, src = video.read()
    if not ok:
        break

    # Get the points for the homography
    merge = merge_views(src, dst, src_list, dst_list)  
    # Display homography   
    cv2.imshow("merge", merge)
    # Store frames
    merged_frames.append(merge)

    # Exit if the 'q' key is pressed
    if cv2.waitKey(500) & 0xFF == ord('q'):
        break

    i+=1

# Release the video capture object and close windows
video.release()
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

...Merging views
Wait for the animation of the tracking. Then press Enter to finish.
H:
[[ 7.00464539e-01  1.00915288e+00  1.36703243e+02]
 [ 2.49445866e-02  1.00812757e+00 -4.98544280e+02]
 [ 6.53228990e-04  2.10287297e-03  1.00000000e+00]]
H:
[[ 7.00464539e-01  1.00915288e+00  1.36703243e+02]
 [ 2.49445866e-02  1.00812757e+00 -4.98544280e+02]
 [ 6.53228990e-04  2.10287297e-03  1.00000000e+00]]
H:
[[ 7.00464539e-01  1.00915288e+00  1.36703243e+02]
 [ 2.49445866e-02  1.00812757e+00 -4.98544280e+02]
 [ 6.53228990e-04  2.10287297e-03  1.00000000e+00]]
H:
[[ 7.00464539e-01  1.00915288e+00  1.36703243e+02]
 [ 2.49445866e-02  1.00812757e+00 -4.98544280e+02]
 [ 6.53228990e-04  2.10287297e-03  1.00000000e+00]]
H:
[[ 7.00464539e-01  1.00915288e+00  1.36703243e+02]
 [ 2.49445866e-02  1.00812757e+00 -4.98544280e+02]
 [ 6.53228990e-04  2.10287297e-03  1.00000000e+00]]
H:
[[ 7.00464539e-01  1.00915288e+00  1.36703243e+02]
 [ 2.49445866e-02  1.00812757e+00 -4.98544280e+02]
 [ 6.53228990e-04  2.1028

-1

In [10]:
from tools import to_gif

to_gif(merged_frames,int(1000/frame_rate), homography_name)

./homography/homography_hockey_static.gif saved!


### 2.2. Written assignment

Analyse the correctness of the predictions and the confidence scores. Try to establish a threshold. Write down your reflections and conclusions.

*answer here*

## Task 3: Homography of a video with moving camera


### 3.1. Code assignment

To account for the moving camera, we perform object tracking of the points used in the homography and recompute the matrix for every frame. Run the code that does point tracking and homography computation. Do it for both hockey and football.  
Save your results as './homography/homography_hockey_dynamic.gif' and './homography/homography_football_dynamic.gif'.

In [19]:
# Globals

drawing = False  # true if mouse is pressed
src_x, src_y = -1, -1
dst_x, dst_y = -1, -1

src_list = []
dst_list = []

# Define the frame and field file paths for the homography
video_path = './homography/video_hockey.mp4'
field_path = './homography/field_hockey.jpg'
homography_name = './homography/homography_hockey_dynamic.gif'

The code below creates a OpenCV tracker (and it's the piece that will fail without opencv-contrib-python).  
You can test the 3 different types and read more about them [here](https://learnopencv.com/object-tracking-using-opencv-cpp-python/). I found KCF to be the best in this application.

In [12]:
# Create the tracker
# see https://github.com/spmallick/learnopencv/blob/master/tracking/tracker.py
# and https://learnopencv.com/object-tracking-using-opencv-cpp-python/
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
tracker_types = ['MIL','KCF','CSRT']
tracker_type = tracker_types[1]
def create_tracker(tracker_type):
    if int(minor_ver) < 3:
        tracker = cv2.Tracker_create(tracker_type)
    else:
        if tracker_type == 'MIL':
            tracker = cv2.TrackerMIL_create()
        if tracker_type == 'KCF':
            tracker = cv2.TrackerKCF_create()
        if tracker_type == "CSRT":
            tracker = cv2.TrackerCSRT_create()
    return tracker

In [13]:
# Mouse callback functions
def select_points_dst(event, x, y, flags, param, dst_copy):
    '''mouse callback function'''
    global dst_x, dst_y, drawing
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        dst_x, dst_y = x, y
        cv2.circle(dst_copy, (x, y), 5, (0, 0, 255), -1)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False

In [14]:
# Merging functions
def get_plan_view(src, dst, src_list, dst_list):
    src_pts = np.array(src_list).reshape(-1, 1, 2)
    dst_pts = np.array(dst_list).reshape(-1, 1, 2)
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    print("H:")
    print(H)
    plan_view = cv2.warpPerspective(src, H, (dst.shape[1], dst.shape[0]))
    return plan_view


def merge_views(src, dst, src_list, dst_list):
    plan_view = get_plan_view(src, dst, src_list, dst_list)
    for i in range(0, dst.shape[0]):
        for j in range(0, dst.shape[1]):
            if (plan_view.item(i, j, 0) == 0 and
               plan_view.item(i, j, 1) == 0 and
               plan_view.item(i, j, 2) == 0):
                plan_view.itemset((i, j, 0), dst.item(i, j, 0))
                plan_view.itemset((i, j, 1), dst.item(i, j, 1))
                plan_view.itemset((i, j, 2), dst.item(i, j, 2))
    return plan_view

In [15]:
# Track funtion to track the points for the frame-by-frame homography
def track_point(video_path, tracker):

    centers = []
    field_centers = []
    video = cv2.VideoCapture(video_path)

    # Exit if video not opened.
    if not video.isOpened():
        print ("Could not open video")
        sys.exit()

    # Read first frame.
    ok, frame = video.read()
    if not ok:
        print ('Cannot read video file')
        sys.exit()

    # Select bounding box
    bbox = cv2.selectROI(frame, False)

    # Draw center point
    center_point_x = int(bbox[0]+ 0.5*bbox[2])
    center_point_y = int(bbox[1] + 0.5*bbox[3])
    center = (center_point_x,center_point_y)
    cv2.circle(frame, center, 2, (0,0,255), -1)
    # Store center point
    centers.append(center)

    # Pause 1000 ms
    key = cv2.waitKey(1000)
    print("Wait for the animation of the tracking. Then press Enter to continue.")

    # Initialize tracker with first frame and bounding box
    ok = tracker.init(frame, bbox)

    while True:
        # Read a new frame
        ok, frame = video.read()
        if not ok:
            break
            
        # Start timer
        timer = cv2.getTickCount()

        # Update tracker
        ok, bbox = tracker.update(frame)

        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)

        # Draw bounding box
        if ok:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
            # Draw center point
            center_point_x = int(bbox[0]+ 0.5*bbox[2])
            center_point_y = int(bbox[1] + 0.5*bbox[3])
            center = (center_point_x,center_point_y)
            cv2.circle(frame, center, 2, (0,0,255), -1)
            # Store center point
            centers.append(center)
        else :
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

        # Display tracker type on frame
        cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2) 
        
        # Display FPS on frame
        cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2) 

        # Display result
        cv2.imshow("video", frame)

        # Exit if ESC pressed
        k = cv2.waitKey(200) & 0xff
        if k == 27 or k == ord('q'): break

    # Release the video capture object and close windows
    video.release()
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

    print(f'centers: {centers}')

    # Open field
    dst = cv2.imread(field_path, -1)
    dst_copy = dst.copy()
    cv2.namedWindow('dst')
    cv2.moveWindow("dst", 780, 80)
    callback = partial(select_points_dst, dst_copy=dst_copy)
    cv2.setMouseCallback('dst', callback)
    
    print("Single click to select the point in the field that best matches the tracked point. Press Enter to continue.")
    
    while True:
        cv2.imshow('dst',dst_copy)
        k = cv2.waitKey(1) & 0xFF
        if k == 13 and (dst_x != -1 and dst_y != -1):
            cv2.circle(dst_copy, (dst_x, dst_y), 5, (0, 255, 0), -1)
            field_center_list = [(dst_x, dst_y)]*len(centers)
            field_centers.extend(field_center_list)
            print(f'field centers: {field_centers}')
            cv2.waitKey(200)  # Wait for 500ms
            break
        elif k == 27 or k == ord('q'):
            break

    print("Press Enter to continue")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

    return centers, field_centers



When you run the code below, a window will pop up with the first frame of the video. You should select the region of interest by drawing a rectangle from the top left to the bottom right corner. You can re-draw it multiple times. When happy with the result, press Enter and the tracking will start. Wait for the animation to finish and make sure the point has been tracked in every frame of the video. Then, press Enter to continue.  
A new window will pop up with the field. Single click the field to select the point in the field that best matches the tracked point. Press Enter to save the point.  
Press Enter to continue to the next point until n_points, where n>=4 for the homography.
When finished, press any key on your keyboard to close the window and release the resources to be able to run the code again, or continue running the next blocks (you need to have the video window selected on that step).

In [16]:
# Run the detection and homography
n_points = 4
centers_h = []
field_centers_h = []

for points in range(n_points):

    tracker = create_tracker(tracker_type)
    centers, field_centers = track_point(video_path, tracker)
    centers_h.append(centers)
    field_centers_h.append(field_centers)


Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Wait for the animation of the tracking. Then press Enter to continue.
centers: [(1701, 815), (1701, 815), (1690, 816), (1677, 815), (1664, 814), (1649, 813), (1640, 814), (1627, 813), (1616, 814), (1605, 813), (1594, 812), (1581, 811), (1572, 810), (1559, 809), (1548, 810), (1539, 809)]
Single click to select the point in the field that best matches the tracked point. Press Enter to continue.
field centers: [(562, 95), (562, 95), (562, 95), (562, 95), (562, 95), (562, 95), (562, 95), (562, 95), (562, 95), (562, 95), (562, 95), (562, 95), (562, 95), (562, 95), (562, 95), (562, 95)]
Press Enter to continue
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Wait for the animation of the tracking. Then press Enter to continue.
centers: [(849, 943), (849, 943), (838, 942), (825, 941), (814, 942), (801, 941), (790, 942), (777, 941), (768, 94

In [17]:
# Change shapes to (frame, point, coordinates)
centers_h_arr = np.array(centers_h)
centers_h_arr = centers_h_arr.transpose(1,0,2)
print(centers_h_arr.shape)
field_centers_h_arr = np.array(field_centers_h)
field_centers_h_arr = field_centers_h_arr.transpose(1,0,2)
print(field_centers_h_arr.shape)

(16, 4, 2)
(16, 4, 2)


The code below iterates the frames of the video and uses the previous caluclated homography matrices to perform the homography frame by frame.

In [18]:
# Perform and display the homography from the saved points
print('...Merging views')
# Store merged frames
merged_frames = []
# Read video
video = cv2.VideoCapture(video_path)
# Get frame count and rate
frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = int(video.get(cv2.CAP_PROP_FPS))
# Read field image
dst = cv2.imread(field_path, -1)
print("Wait for the animation of the tracking. Then press Enter to finish.")

i=0
while True:
    # Exit if video not opened.
    if not video.isOpened():
        print ("Could not open video")
        sys.exit()

    # Read video frame.
    ok, src = video.read()
    if not ok:
        break

    # Get the points for the homography
    src_list = centers_h_arr[i].tolist()
    dst_list = field_centers_h_arr[i].tolist()
    merge = merge_views(src, dst, src_list, dst_list)  
    # Display homography   
    cv2.imshow("merge", merge)
    # Store frames
    merged_frames.append(merge)

    # Exit if the 'q' key is pressed
    if cv2.waitKey(200) & 0xFF == ord('q'):
        break

    i+=1

# Release the video capture object and close windows
video.release()
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

...Merging views
Wait for the animation of the tracking. Then press Enter to finish.
H:
[[ 5.36659364e-01  7.60658986e-01  2.05306581e+02]
 [ 1.05055742e-02  7.83107673e-01 -3.62295587e+02]
 [ 4.69938737e-04  1.58692151e-03  1.00000000e+00]]
H:
[[ 5.36659364e-01  7.60658986e-01  2.05306581e+02]
 [ 1.05055742e-02  7.83107673e-01 -3.62295587e+02]
 [ 4.69938737e-04  1.58692151e-03  1.00000000e+00]]
H:
[[ 5.59454424e-01  7.85975893e-01  2.02405865e+02]
 [ 1.09388343e-02  8.03896213e-01 -3.72014310e+02]
 [ 4.98947537e-04  1.64274514e-03  1.00000000e+00]]
H:
[[ 5.62705255e-01  7.99092501e-01  2.04345103e+02]
 [ 1.10323122e-02  8.10830293e-01 -3.75182130e+02]
 [ 5.01771805e-04  1.66878878e-03  1.00000000e+00]]
H:
[[ 5.47156982e-01  7.74350442e-01  2.14545556e+02]
 [ 1.15210408e-02  7.91238690e-01 -3.66518802e+02]
 [ 4.84344569e-04  1.61845745e-03  1.00000000e+00]]
H:
[[ 5.37379060e-01  7.58913519e-01  2.21799314e+02]
 [ 1.13280026e-02  7.79740074e-01 -3.61027174e+02]
 [ 4.73561699e-04  1.5847

-1

In [20]:
from tools import to_gif

to_gif(merged_frames,int(1000/frame_rate), homography_name)

./homography/homography_hockey_dynamic.gif saved!


### 3.2. Code assignment

Choose another sport where you think the same technique could work, find a picture of the field and a short video of a play, and perform the homography. Save your results as './homography/homography_sport_dynamic.gif

### 3.3. Written assignment

Think about how this technology could be applied in the sport that you chose on task 3.2. Write a paragraph with your reflections.

*answer here*

## Bonus Task 1



(For all)

Now you are already familiar with multiple tracking algorithms, choose one of the videos provided (hockey or american football) and perform the homography while tracking the position of one or more players. Plot the positions of the players that you track on the image of the field as dots of different colors. Save a gif displaying the dots moving on the field as './homography/bonus_task.gif'.  